In [ ]:
import heapq
from pprint import pprint
import random

In [ ]:
def dijkstra(grafo, origem):
    # Calcula a menor distância de um ponto de origem para todos os outros
    dist = {nodo: float('inf') for nodo in grafo}
    dist[origem] = 0
    fila = [(0, origem)]  # (distância, nodo)

    while fila:
        dist_atual, nodo_atual = heapq.heappop(fila)

        if dist_atual > dist[nodo_atual]:
            continue

        for vizinho, peso in grafo[nodo_atual]:
            nova_dist = dist_atual + peso

            if nova_dist < dist[vizinho]:
                dist[vizinho] = nova_dist
                heapq.heappush(fila, (nova_dist, vizinho))

    return dist

In [ ]:
# Listas
cds = ['CD1', 'CD2', 'CD3', 'CD4', 'CD5'] # Centro de distribuição

In [ ]:
from datetime import datetime, timedelta
import math

def calc_hours_diff(future_date):
    current_datetime = datetime.now()
    hours_diff = (future_date - current_datetime).total_seconds() / 3600
    return math.ceil(hours_diff)

# Função para gerar encomendas randomicamente
def gerar_encomenda_randomica(id_encomenda):
    destino = random.choice(['D1', 'D2', 'D3'])
    prazo_entrega = calc_hours_diff(datetime.now() + timedelta(days=random.randint(1, 10)))  # Dias
    peso = random.randint(1000, 10000)  # Kgs

    return {
        'id_encomenda': id_encomenda,
        'id_destino': destino,
        'prazo_entrega': prazo_entrega,
        'peso': peso
    }


encomendas = []
# Gerar múltiplas encomendas
for i in range(20):
    id_encomenda = f"E{i+1}"
    encomendas.append(gerar_encomenda_randomica(id_encomenda))

# Ordena pelo prazo de entrega
encomendas.sort(key=lambda x: x["prazo_entrega"])


raw_encomendas = encomendas.copy()

pprint(encomendas)

[{'id_destino': 'D3', 'id_encomenda': 'E5', 'peso': 6541, 'prazo_entrega': 24},
 {'id_destino': 'D2', 'id_encomenda': 'E14', 'peso': 2033, 'prazo_entrega': 24},
 {'id_destino': 'D3', 'id_encomenda': 'E20', 'peso': 3772, 'prazo_entrega': 24},
 {'id_destino': 'D3', 'id_encomenda': 'E6', 'peso': 8607, 'prazo_entrega': 96},
 {'id_destino': 'D3', 'id_encomenda': 'E16', 'peso': 1179, 'prazo_entrega': 96},
 {'id_destino': 'D2', 'id_encomenda': 'E2', 'peso': 6617, 'prazo_entrega': 120},
 {'id_destino': 'D3', 'id_encomenda': 'E4', 'peso': 7484, 'prazo_entrega': 120},
 {'id_destino': 'D1', 'id_encomenda': 'E9', 'peso': 5378, 'prazo_entrega': 120},
 {'id_destino': 'D1', 'id_encomenda': 'E7', 'peso': 9655, 'prazo_entrega': 144},
 {'id_destino': 'D1',
  'id_encomenda': 'E19',
  'peso': 4040,
  'prazo_entrega': 168},
 {'id_destino': 'D3',
  'id_encomenda': 'E10',
  'peso': 7482,
  'prazo_entrega': 192},
 {'id_destino': 'D3',
  'id_encomenda': 'E15',
  'peso': 3205,
  'prazo_entrega': 192},
 {'id_des

In [ ]:
# Função para gerar caminhoes randomicamente
def gerar_caminhao_randomico(id_caminhao, id_cd):
    limite_operacional = random.randint(4, 14)  # Horas
    peso_maximo = random.randint(1000, 10000)  # Kgs

    return {
        'id_caminhao': id_caminhao,
        'id_cd': id_cd,
        'peso_maximo': peso_maximo,
        'limite_operacional': limite_operacional
    }


caminhoes_list = []
# Garante que cada CD tenha pelo menos um caminhão
for i, cd in enumerate(cds):
    id_caminhao = f"C{i+1:03}"
    caminhoes_list.append(gerar_caminhao_randomico(id_caminhao, cd))

# Gera caminhões adicionais aleatórios
num_caminhoes_adicionais = 2
for i in range(len(caminhoes_list), len(caminhoes_list) + num_caminhoes_adicionais):
    id_caminhao = f"C{i+1:03}"
    cd_random = random.choice(cds)
    caminhoes_list.append(gerar_caminhao_randomico(id_caminhao, cd_random))


# Dicionário
caminhoes = {}

# transforma lista para hash
for caminhao in caminhoes_list:
    if caminhao['id_cd'] not in caminhoes:
        caminhoes[caminhao['id_cd']] = []
    caminhoes[caminhao['id_cd']].append(caminhao)

pprint(caminhoes)

{'CD1': [{'id_caminhao': 'C001',
          'id_cd': 'CD1',
          'limite_operacional': 13,
          'peso_maximo': 9788}],
 'CD2': [{'id_caminhao': 'C002',
          'id_cd': 'CD2',
          'limite_operacional': 11,
          'peso_maximo': 5037},
         {'id_caminhao': 'C007',
          'id_cd': 'CD2',
          'limite_operacional': 4,
          'peso_maximo': 1456}],
 'CD3': [{'id_caminhao': 'C003',
          'id_cd': 'CD3',
          'limite_operacional': 14,
          'peso_maximo': 1407}],
 'CD4': [{'id_caminhao': 'C004',
          'id_cd': 'CD4',
          'limite_operacional': 13,
          'peso_maximo': 1934},
         {'id_caminhao': 'C006',
          'id_cd': 'CD4',
          'limite_operacional': 6,
          'peso_maximo': 4688}],
 'CD5': [{'id_caminhao': 'C005',
          'id_cd': 'CD5',
          'limite_operacional': 9,
          'peso_maximo': 4079}]}


In [ ]:
# (vizinho, distancia em horas)
grafo = {
    'CD1': [('D1', 4), ('D2', 10), ('D3', 2)],
    'CD2': [('D1', 10), ('D2', 3), ('D3', 7)],
    'CD3': [('D1', 5), ('D2', 8), ('D3', 10)],
    'CD4': [('D1', 2), ('D2', 8), ('D3', 8)],
    'CD5': [('D1', 6), ('D2', 6), ('D3', 6)],
    'D1': [], 'D2': [], 'D3': []  # Destinos não têm arestas saindo deles
}

In [ ]:
# Calcula as menores distâncias de cada CD para todos os destinos
resultados = {}
for cd in cds:
    resultados[cd] = dijkstra(grafo, cd)

print(resultados)

{'CD1': {'CD1': 0, 'CD2': inf, 'CD3': inf, 'CD4': inf, 'CD5': inf, 'D1': 4, 'D2': 10, 'D3': 2}, 'CD2': {'CD1': inf, 'CD2': 0, 'CD3': inf, 'CD4': inf, 'CD5': inf, 'D1': 10, 'D2': 3, 'D3': 7}, 'CD3': {'CD1': inf, 'CD2': inf, 'CD3': 0, 'CD4': inf, 'CD5': inf, 'D1': 5, 'D2': 8, 'D3': 10}, 'CD4': {'CD1': inf, 'CD2': inf, 'CD3': inf, 'CD4': 0, 'CD5': inf, 'D1': 2, 'D2': 8, 'D3': 8}, 'CD5': {'CD1': inf, 'CD2': inf, 'CD3': inf, 'CD4': inf, 'CD5': 0, 'D1': 6, 'D2': 6, 'D3': 6}}


In [ ]:
# Ordena o cds do mais proximo ao mais distante em relação ao destino
def ordera_cds_por_distancia(resultados, destino):
    cds_to_destino = []
    for cd in resultados:
        cds_to_destino.append((cd, resultados[cd][destino]))
    return sorted(cds_to_destino, key=lambda x: x[1])

In [ ]:
import sys
# Armazena o resultado de cada encomenda
entregas = []
caminhoes_ocupados = [];

encomendas = raw_encomendas.copy()

while encomendas:
    if(len(encomendas) == 0): raise IndexError("A fila está vazia")
    encomenda = encomendas.pop(0)

    distancias_cd_destino = ordera_cds_por_distancia(resultados, encomenda['id_destino'])

    for(cd, distancia) in distancias_cd_destino:
        if any(encomenda['id_encomenda'] == entrega['id_encomenda'] for entrega in entregas): break # Verifica se a encomenta já foi adicionada a lista de entregas

        for caminhao in caminhoes[cd]:
            if(encomenda['prazo_entrega'] < distancia): #caso o prazo não possa ser atendido
                print(f"A encomenda {encomenda} não pode ser entregue no prazo")
                sys.exit()
            if(caminhao['peso_maximo'] < encomenda['peso'] or caminhao['id_caminhao'] in caminhoes_ocupados): continue # Adicionar a validação de prazo de entrega

            entregas.append({
                'id_encomenda': encomenda['id_encomenda'],
                'id_caminhao': caminhao['id_caminhao'],
                'distancia': distancia,
                'prazo_entrega': encomenda['prazo_entrega'],
                'destino': encomenda['id_destino'],
                'peso': encomenda['peso']
            })
            caminhoes_ocupados.append(caminhao['id_caminhao'])
            break
print(entregas)

[{'id_encomenda': 'E5', 'id_caminhao': 'C001', 'distancia': 2, 'prazo_entrega': 24, 'destino': 'D3', 'peso': 6541}, {'id_encomenda': 'E14', 'id_caminhao': 'C002', 'distancia': 3, 'prazo_entrega': 24, 'destino': 'D2', 'peso': 2033}, {'id_encomenda': 'E20', 'id_caminhao': 'C005', 'distancia': 6, 'prazo_entrega': 24, 'destino': 'D3', 'peso': 3772}, {'id_encomenda': 'E16', 'id_caminhao': 'C007', 'distancia': 7, 'prazo_entrega': 96, 'destino': 'D3', 'peso': 1179}, {'id_encomenda': 'E19', 'id_caminhao': 'C006', 'distancia': 2, 'prazo_entrega': 168, 'destino': 'D1', 'peso': 4040}, {'id_encomenda': 'E17', 'id_caminhao': 'C004', 'distancia': 2, 'prazo_entrega': 216, 'destino': 'D1', 'peso': 1107}]


In [ ]:
from pprint import pprint
for entrega in entregas:
    print(f"encomenda {entrega['id_encomenda']}({entrega['peso']} kg) -> caminhão {entrega['id_caminhao']} em {entrega['distancia']} horas.")

pprint(entregas)

encomenda E5(6541 kg) -> caminhão C001 em 2 horas.
encomenda E14(2033 kg) -> caminhão C002 em 3 horas.
encomenda E20(3772 kg) -> caminhão C005 em 6 horas.
encomenda E16(1179 kg) -> caminhão C007 em 7 horas.
encomenda E19(4040 kg) -> caminhão C006 em 2 horas.
encomenda E17(1107 kg) -> caminhão C004 em 2 horas.
[{'destino': 'D3',
  'distancia': 2,
  'id_caminhao': 'C001',
  'id_encomenda': 'E5',
  'peso': 6541,
  'prazo_entrega': 24},
 {'destino': 'D2',
  'distancia': 3,
  'id_caminhao': 'C002',
  'id_encomenda': 'E14',
  'peso': 2033,
  'prazo_entrega': 24},
 {'destino': 'D3',
  'distancia': 6,
  'id_caminhao': 'C005',
  'id_encomenda': 'E20',
  'peso': 3772,
  'prazo_entrega': 24},
 {'destino': 'D3',
  'distancia': 7,
  'id_caminhao': 'C007',
  'id_encomenda': 'E16',
  'peso': 1179,
  'prazo_entrega': 96},
 {'destino': 'D1',
  'distancia': 2,
  'id_caminhao': 'C006',
  'id_encomenda': 'E19',
  'peso': 4040,
  'prazo_entrega': 168},
 {'destino': 'D1',
  'distancia': 2,
  'id_caminhao': 